In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import re
import json

Using TensorFlow backend.
/Users/prajwal967/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/prajwal967/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:472: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/prajwal967/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:473: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/prajwal967/anaconda3/lib/python3.6

In [2]:
from corpus import ParseTweets, BuildCorpus

In [3]:
class LTSM():
    
    def __init__(self):
        self.model = Sequential()
        
    def build_model(self, corpus):
        self.model.add(LSTM(128, input_shape = (corpus.maxlen, len(corpus.chars))))
        self.model.add(Dense(len(corpus.chars)))
        self.model.add(Activation('softmax'))
        optimizer = RMSprop(lr = 0.01)
        self.model.compile(loss = 'categorical_crossentropy', optimizer = optimizer)
        
    def sample(self, preds, temperature = 1.0):
        # helper function to sample an index from a probability array
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)
        
    def train(self, X, Y, corpus):
        
        # train the model, output generated text after each iteration
        for iteration in range(1, 10):
            print()
            print('-' * 50)
            print('Iteration', iteration)
            self.model.fit(X, Y, batch_size = 128, epochs = 10)

            start_index = random.randint(0, len(corpus.text) - corpus.maxlen - 1)

            for diversity in [0.2, 0.5, 1.0, 1.2]:
                print()
                print('----- diversity:', diversity)

                generated = ''
                sentence = corpus.text[start_index: start_index + corpus.maxlen]
                generated += sentence
                print('----- Generating with seed: "' + sentence + '"')
                print('\nGenerated')
                sys.stdout.write(generated)

                for i in range(20):
                    x = np.zeros((1, corpus.maxlen, len(corpus.chars)))
                    for t, char in enumerate(sentence):
                        x[0, t, corpus.char_indices[char]] = 1.

                    preds = self.model.predict(x, verbose=0)[0]
                    next_index = self.sample(preds, diversity)
                    next_char = corpus.indices_char[next_index]

                    generated += next_char
                    sentence = sentence[1:] + next_char

                    sys.stdout.write(next_char)
                    sys.stdout.flush()
                print()

In [4]:
if __name__ == "__main__":
    
    parse_tweets = ParseTweets()
    text = parse_tweets.parse_tweets('euro_python.json', False)
    corpus = BuildCorpus(text)
    corpus.build_sentences()
    X, Y = corpus.vectorize_text()
    
    ltsm = LTSM()
    ltsm.build_model(corpus)
    ltsm.train(X, Y, corpus)


--------------------------------------------------
Iteration 1
Epoch 1/10
12555/12555 [==============================] - 8s 659us/step - loss: 3.0147
Epoch 2/10
12555/12555 [==============================] - 8s 652us/step - loss: 2.5031
Epoch 3/10
12555/12555 [==============================] - 8s 640us/step - loss: 2.1928
Epoch 4/10
12555/12555 [==============================] - 8s 671us/step - loss: 1.9030
Epoch 5/10
12555/12555 [==============================] - 8s 631us/step - loss: 1.6311
Epoch 6/10
12555/12555 [==============================] - 8s 623us/step - loss: 1.3952
Epoch 7/10
12555/12555 [==============================] - 8s 649us/step - loss: 1.1991
Epoch 8/10
12555/12555 [==============================] - 8s 640us/step - loss: 1.0386
Epoch 9/10
12555/12555 [==============================] - 8s 632us/step - loss: 0.9217
Epoch 10/10
12555/12555 [==============================] - 8s 633us/step - loss: 0.8126

----- diversity: 0.2
----- Generating with seed: "s wifi support

12555/12555 [==============================] - 8s 666us/step - loss: 0.1701
Epoch 3/10
12555/12555 [==============================] - 8s 641us/step - loss: 0.1651
Epoch 4/10
12555/12555 [==============================] - 8s 644us/step - loss: 0.1729
Epoch 5/10
12555/12555 [==============================] - 8s 650us/step - loss: 0.1599
Epoch 6/10
12555/12555 [==============================] - 8s 627us/step - loss: 0.1569
Epoch 7/10
12555/12555 [==============================] - 8s 654us/step - loss: 0.1601
Epoch 8/10
12555/12555 [==============================] - 9s 720us/step - loss: 0.1608
Epoch 9/10
12555/12555 [==============================] - 9s 706us/step - loss: 0.1629
Epoch 10/10
12555/12555 [==============================] - 9s 753us/step - loss: 0.1608

----- diversity: 0.2
----- Generating with seed: " … compa ñera @claudiaguirao estar á tod"

Generated
 … compa ñera @claudiaguirao estar á toda la seming sleres o

----- diversity: 0.5
----- Generating with seed: " … compa ñe

/Users/prajwal967/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


12555/12555 [==============================] - 8s 657us/step - loss: 0.1609
Epoch 2/10
12555/12555 [==============================] - 8s 661us/step - loss: 0.1612
Epoch 3/10
12555/12555 [==============================] - 8s 641us/step - loss: 0.1521
Epoch 4/10
12555/12555 [==============================] - 8s 641us/step - loss: 0.1558
Epoch 5/10
12555/12555 [==============================] - 8s 642us/step - loss: 0.1551
Epoch 6/10
12555/12555 [==============================] - 9s 721us/step - loss: 0.1410
Epoch 7/10
12555/12555 [==============================] - 9s 712us/step - loss: 0.1498
Epoch 8/10
12555/12555 [==============================] - 9s 682us/step - loss: 0.1417
Epoch 9/10
12555/12555 [==============================] - 8s 654us/step - loss: 0.1402
Epoch 10/10
12555/12555 [==============================] - 10s 794us/step - loss: 0.1342

----- diversity: 0.2
----- Generating with seed: "rowds every day and are one of my favori"

Generated
rowds every day and are one of my f